In [1]:
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import tkinter as tk
from pysrc.simplertplot.plotwidget import RTPlotWidget
from pysrc.simplertplot import queues
from hello.hello3 import HelloApp, TrueError
from time import time, sleep
import numpy as np
from matplotlib.ticker import NullFormatter, NullLocator, FormatStrFormatter

In [3]:
#ip = '192.168.1.85'
ip = '71.189.82.196:85'

In [4]:
interval = 1000
minutes = 60
max_pts=minutes*60/(interval/1000)
max_pts=int(max_pts)
max_pts

3600

In [5]:
class MyPlotWidget(RTPlotWidget):
    def __init__(self, *args, **kw):
        super().__init__(*args, **kw)
        self._special()
    def _ma(self, a, n):
        rv = np.zeros(len(a))
        n //= 2
        if n < 1:
            n = 1
        for i in range(len(a)):
            rv[i] = np.mean(a[max(i-n, 0):i+n])
        return rv

    def _update_data(self):

#         _plot_x_data = self._ma(self.x_data.get(), 3)
#         _plot_y_data = self._ma(self.y_data.get(), 3)
#         _plot_y2_data = self._ma(self.y_data2.get(), 3)
        self.line.set_data(self.x_data.get(), self.y_data.get())
        self.line2.set_data(self.x_data.get(), self.y_data2.get())
        self.subplot.relim()
        self.subplot.autoscale_view(True, True, True)
        self.ax2.relim()
        self.ax2.autoscale_view(True,True,True)
        lower, upper = self.subplot.get_ybound()
        self.subplot.set_ylim(lower-0.2, upper+0.2, True, None)
        lower, upper = self.ax2.get_ybound()
        self.ax2.set_ylim(lower-1, upper+1, True, None)
        self._update()
        
    def _special(self):
        self.ax2 = self.subplot.twinx()
        self.y_data2 = queues.RingBuffer(self.max_pts)
        self.line2, = self.ax2.plot(self.x_data.get(), self.y_data2.get(), 'g-')
        self.yaxis2 = self.ax2.yaxis
        self.yaxis2.set_major_formatter(FormatStrFormatter("%.2f"))

    def extend_xy(self, x, y, y2):
        self.x_data.extend(x)
        self.y_data.extend(y)
        self.y_data2.extend(y2)
        self._update_data()
        
    def reset(self):
        self.x_data.clear()
        self.y_data.clear()
        self.y_data2.clear()
        self._update_data()

    def add_xy(self, x, y, y2):
        self.x_data.put(x)
        self.y_data.put(y)
        self.y_data2.put(y2)
        self._update_data()

In [6]:
def timer():
    start = time()
    while True:
        current = time()
        yield current - start
get_time = timer().__next__

In [7]:
g_stop_tasks = False
g_after = None
def tk_task(tk, interval, f):
    global g_after, g_stop_tasks
    def fn(*args, **kw):
        global g_after
        try:
            return f(*args, **kw)
        finally:
            if not g_stop_tasks:
                g_after = tk.after(interval, fn)
    g_after = tk.after(interval, fn)

In [8]:
controller = 'temperature'

In [10]:
import threading
class Model():
    def __init__(self, app, ctrl, interval=0.1):
        self.mv = app.getMainValues()  # prepopulate
        self.time = get_time()
        self.lock = threading.RLock()
        self.running = threading.Event()
        self.thread = None
        self.app = app
        self.interval = interval
        self.ctrl = ctrl
    def getvalues(self):
        with self.lock:
            pv = self.mv[self.ctrl]['pv']
            op = self.mv[self.ctrl]['output']
            t = self.time
        return t, pv, op
    def getMainValues(self):
        return self.mv
    def run_update(self):
        self.running.set()
        self.thread = threading.Thread(None, self._run_update, daemon=True)
        self.thread.start()
    def _run_update(self):
        while self.running.is_set():
            try:
                mv = self.app.getMainValues()
            except Exception:
                self.app.reconnect()
                continue
            with self.lock:
                self.mv = mv
                self.time = get_time()
            sleep(self.interval)
    def stop(self):
        self.running.clear()
        self.thread.join(timeout=10)
        if self.thread.is_alive():
            raise RuntimeError("Failed to stop thread")
    def reset(self):
        self.time = get_time()
        self.stop()
        self.run_update()

h = HelloApp(ip)
m = Model(h, controller, 0.5)
m.run_update()

def update():
    t, pv, op = m.getvalues()
    p.add_xy(t, pv, op)
    update_labels(pv, op)

In [11]:
def printdir(o):
    for a in dir(o):
        if not a.startswith("_"):
            print(a, getattr(o, a))

In [12]:
def conf2(ev):
    if ev.widget == root:
        p.tkcanvas.config(height=ev.height, width=ev.width-rbtn.winfo_width())

In [13]:
root = tk.Tk()
root.withdraw()

p = MyPlotWidget(root, max_pts=max_pts)
p.tkcanvas.config(height=400, width=600)
max_pts


3600

In [14]:
class ILabel(tk.Label):
    def __init__(self, master, name, value=0, fmt="%.2f"):
        super().__init__(master)
        self.fmt = fmt
        self.name = name
        self.set_value(value)
        
    def set_value(self, val):
        val = self.fmt % val
        self.config(text="%s: %s" % (self.name, val))
   
labels = {}
def mk_label(name, val, row):
    l = ILabel(root, name, val)
    labels[name] = l
    l.grid(row=row, column=1)
    
mk_label("PV", 0, 1)
mk_label("OP", 0, 2)
    
def update_labels(pv, op):
    labels['PV'].set_value(pv)
    labels['OP'].set_value(op)

    

In [15]:
p.grid(rowspan=20, column=0, row=0)    
def reset():
    p.reset()
    global get_time
    get_time = timer().__next__
    m.reset()
rbtn = tk.Button(root, text="Reset", command=reset)
rbtn.grid(column=1, row=0, sticky="ew")
root.bind("<Configure>", conf2)
p.set_xaxis_text("Time (s)")
p.set_yaxis_text("%s PV" % controller.title())
def on_closing():
    from tkinter import messagebox
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        global g_stop_tasks, g_after
        g_stop_tasks = True
        root.after_cancel(g_after)
        root.quit()
        root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.wm_deiconify()
tk_task(root, interval, update)
try:
    root.mainloop()
finally:
    m.stop()


ERROR OCCURRED DURING REQUEST
IPV4 ADDRESS: 71.189.82.196:85
REQUESTED URL: </webservice/interface/?&call=getMainValues&json=true>
MESSAGE: ConnectionError(('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)))
ERROR OCCURRED DURING REQUEST
IPV4 ADDRESS: 71.189.82.196:85
REQUESTED URL: </webservice/interface/?&call=getMainValues&json=true>
MESSAGE: ConnectionError(HTTPSConnectionPool(host='71.189.82.196', port=85): Max retries exceeded with url: /webservice/interface/?&call=getMainValues&json=true (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x00000238B913B5C0>: Failed to establish a new connection: [WinError 10051] A socket operation was attempted to an unreachable network',)))
ERROR OCCURRED DURING REQUEST
IPV4 ADDRESS: 71.189.82.196:85
REQUESTED URL: </webservice/interface/?&call=getMainValues&json=true>
MESSAGE: ConnectionError(